# Exercise 4

In [12]:
import math
import numpy as np
import heapq
import scipy.stats as stats
import matplotlib.pyplot as plt


In [13]:
def simulate_blocking(ia_times, service_times, m, n_customers):
    busy = 0                 # current busy servers
    blocked = 0              # count of blocked arrivals
    t = 0.0                  # current time
    departures = []          # min-heap of departure times
    for i in range(n_customers):
        t += ia_times[i]     # advance to next arrival
        # free up servers for any departures before t
        while departures and departures[0] <= t:
            heapq.heappop(departures)
            busy -= 1
        # arrival: accept if free server, else block
        if busy < m:
            busy += 1
            # schedule departure
            heapq.heappush(departures, t + service_times[i])
        else:
            blocked += 1
    return blocked / n_customers


## Part 1


In [14]:
# Erlang B formula
def erlangB(A, m):
    num = A**m / math.factorial(m)
    den = sum(A**i / math.factorial(i) for i in range(m+1))
    return num/den

m, mean_svc, A = 10, 8, 8
R, N = 10, 10000  # replications and customers per rep
results1 = []
for _ in range(R):
    ia = np.random.exponential(1, N)
    svc = np.random.exponential(mean_svc, N)
    results1.append(simulate_blocking(ia, svc, m, N))
arr1 = np.array(results1)
mean1 = arr1.mean()
se1 = arr1.std(ddof=1)/np.sqrt(R)
tval = stats.t.ppf(0.975, df=R-1)
ci1 = (mean1 - tval*se1, mean1 + tval*se1)
print(f"Simulated blocking = {mean1:.4f}, 95% CI = [{ci1[0]:.4f}, {ci1[1]:.4f}]")
print(f"Analytical Erlang B = {erlangB(A, m):.4f}")

Simulated blocking = 0.1227, 95% CI = [0.1183, 0.1271]
Analytical Erlang B = 0.1217


## Part 2

In [15]:
def simulate_renewal(arrival_gen, label):
    res = []
    for _ in range(R):
        ia = arrival_gen(N)
        svc = np.random.exponential(mean_svc, N)
        res.append(simulate_blocking(ia, svc, m, N))
    arr = np.array(res)
    mu, se = arr.mean(), arr.std(ddof=1)/np.sqrt(R)
    ci = (mu - tval*se, mu + tval*se)
    print(f"{label}: blocking={mu:.4f}, CI=[{ci[0]:.4f},{ci[1]:.4f}]")

# Erlang shape=2
simulate_renewal(lambda n: stats.gamma.rvs(2, scale=1/2, size=n), 'Erlang(2)')
# Hyper-exponential
def hyper_exp(n):
    u = np.random.rand(n)
    ia = np.empty(n)
    ia[u<0.8] = np.random.exponential(1/0.8333, np.sum(u<0.8))
    ia[u>=0.8] = np.random.exponential(1/5.0, np.sum(u>=0.8))
    return ia
simulate_renewal(hyper_exp, 'Hyper-exp')

Erlang(2): blocking=0.0931, CI=[0.0892,0.0969]
Hyper-exp: blocking=0.1393, CI=[0.1357,0.1428]


## Part 3

In [16]:
def run_service_experiment(gen_svc, label):
    res = []
    for _ in range(R):
        ia = np.random.exponential(1, N)
        svc = gen_svc(N)
        res.append(simulate_blocking(ia, svc, m, N))
    arr = np.array(res)
    mu, se = arr.mean(), arr.std(ddof=1)/np.sqrt(R)
    ci = (mu - tval*se, mu + tval*se)
    print(f"{label}: blocking={mu:.4f}, CI=[{ci[0]:.4f},{ci[1]:.4f}]")

# constant
run_service_experiment(lambda n: np.full(n, mean_svc), 'Constant')
# Pareto helper
def pareto_gen(n, k):
    xm = (k-1)/k * mean_svc
    return stats.pareto.rvs(k, scale=xm, size=n)
run_service_experiment(lambda n: pareto_gen(n, 1.05), 'Pareto k=1.05')
run_service_experiment(lambda n: pareto_gen(n, 2.05), 'Pareto k=2.05')
# uniform
run_service_experiment(lambda n: np.random.rand(n)*16, 'Uniform(0,16)')

Constant: blocking=0.1222, CI=[0.1195,0.1249]
Pareto k=1.05: blocking=0.0016, CI=[0.0008,0.0024]
Pareto k=2.05: blocking=0.1211, CI=[0.1175,0.1247]
Uniform(0,16): blocking=0.1193, CI=[0.1151,0.1235]


## Part 4: Comparison of Confidence Intervals
Differences arise because arrival/service variability affects both blocking probability and its sampling variability:
- Lower variability (Erlang arrivals, constant service) leads to tighter CIs.
- Higher variability (hyper-exp arrivals, heavy-tailed Pareto) leads to wider CIs and usually higher blocking.
